## Veiew DB in panadas

In [2]:
import sqlite3
import pandas as pd

dat = sqlite3.connect('option_strategies.db')
query = dat.execute("SELECT * From option_strategies")
cols = [column[0] for column in query.description]
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
results[['id', 'strategy_type', 'tab_name', 'er', 'ticker','trigger_price', 'strike_price', 'strike_buy','strike_sell','estimated_premium', 'last_price_when_checked',
         'timestamp_of_price_when_last_checked','scrape_date', 'timestamp_of_trigger','strategy_status', 'price_when_triggered', 'price_when_order_placed','premium_when_last_checked']].tail(22)

,id,strategy_type,tab_name,er,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,last_price_when_checked,timestamp_of_price_when_last_checked,scrape_date,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed,premium_when_last_checked
0,1,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,0,AMZN,203.05,sell 212.5 - buy 222.5,222.5,212.5,163.0,None,None,2025-04-03T07:31:37.326720,None,None,None,None,None
1,2,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,0,GOOGL,165.11,sell 172.5 - buy 182.5,182.5,172.5,109.0,None,None,2025-04-03T07:31:37.404734,None,None,None,None,None
2,3,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,0,IWM,205.09,sell 213.0 - buy 223.0,223.0,213.0,116.0,None,None,2025-04-03T07:31:37.486641,None,None,None,None,None
3,4,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,0,SPY,568.04,sell 591.0 - buy 601.0,601.0,591.0,135.0,None,None,2025-04-03T07:31:37.563132,None,None,None,None,None
4,5,Bear Call,Minimal Risk 97-99% accuracy > shorter expiry,0,META,603.55,sell 650.0 - buy 660.0,660.0,650.0,128.0,None,None,2025-04-03T07:31:41.133919,None,None,None,None,None
5,6,Bear Call,Mild Risk 95-97% accuracy > longer expiry,0,None,None,None,0.0,0.0,None,None,None,2025-04-03T07:31:44.686291,None,None,None,None,None
6,7,Bear Call,Minimal Risk 97-99% accuracy > longer expiry,0,None,None,None,0.0,0.0,None,None,None,2025-04-03T07:31:48.245315,None,None,None,None,None
7,8,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,0,JNJ,152.18,sell 147.0 - buy 135.0,135.0,147.0,75.0,None,None,2025-04-03T07:31:57.908374,None,None,None,None,None
8,9,Bull Put,Minimal Risk 97-99% accuracy > shorter expiry,0,None,None,None,0.0,0.0,None,None,None,2025-04-03T07:32:01.549617,None,None,None,None,None
9,10,Bull Put,Mild Risk 95-97% accuracy > longer expiry,0,JNJ,152.56,sell 145.0 - buy 135.0,135.0,145.0,79.0,None,None,2025-04-03T07:32:05.162256,None,None,None,None,None


In [15]:
results.columns

Index(['id', 'scrape_date', 'strategy_type', 'tab_name', 'ticker',
       'trigger_price', 'strike_price', 'strike_buy', 'strike_sell',
       'estimated_premium', 'last_price_when_checked',
       'timestamp_of_price_when_last_checked', 'item_id',
       'options_expiry_date', 'date_info', 'timestamp_of_trigger',
       'strategy_status', 'price_when_triggered', 'price_when_order_placed',
       'premium_when_last_checked', 'timestamp_of_order'],
      dtype='object')

In [18]:
results[~results['timestamp_of_trigger'].isnull()]

,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,premium_received
8,9,2025-03-10T08:41:21.213768,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,WMT,89.68,sell 86.0 - buy 75.0,75.0,86.0,58.0,4,2025-03-21,March,2025-03-10T14:51:27.476449,premium too low,87.97,NaN
12,13,2025-03-10T08:41:28.351534,Bull Put,Mild Risk 95-97% accuracy > longer expiry,META,589.67,sell 540.0 - buy 530.0,530.0,540.0,97.0,2,2025-03-28,March,2025-03-10T14:57:11.292093,premium too low,589.12,NaN
14,15,2025-03-10T08:41:28.538762,Bull Put,Mild Risk 95-97% accuracy > longer expiry,WMT,89.29,sell 86.0 - buy 80.0,80.0,86.0,70.0,4,2025-03-28,March,2025-03-10T14:51:25.356915,premium too low,87.97,NaN
16,17,2025-03-10T08:41:32.218233,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,QQQ,490,sell 460.0 - buy 450.0,450.0,460.0,273.0,2,2025-03-28,March,2025-03-10T08:46:27.463750,premium too low,488.20,NaN
17,18,2025-03-10T08:41:32.354756,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,TQQQ,62.25,sell 60.0 - buy 50.0,50.0,60.0,210.0,3,2025-03-28,March,2025-03-10T14:33:41.525971,premium too low,61.69,NaN
46,47,2025-03-12T06:02:01.555882,Bull Put,Minimal Risk 97-99% accuracy > shorter expiry,JNJ,163.62,sell 157.5 - buy 148.0,148.0,157.5,72.0,1,2025-03-28,March,2025-03-12T21:19:42.678459,order placed,162.70,60.0


## Delete ALL rows in SQLite database

In [6]:

import sqlite3

conn = sqlite3.connect('option_strategies.db')
c = conn.cursor()

# Delete all rows from table
c.execute('DELETE FROM option_strategies;',)

print('We have deleted', c.rowcount, 'records from the table.')

# Commit the changes to db			
conn.commit()
# Close the connection
conn.close()

We have deleted 123 records from the table.


## Delete table

In [19]:

# importing sqlite module 
import sqlite3 
  
# create connection to the  
# database geek 
connection = sqlite3.connect('option_strategies.db') 
  
# drop table 
connection.execute("DROP TABLE option_strategies") 
  
print("data dropped successfully") 
  
# close the connection 
connection.close() 

data dropped successfully


## Update Row

In [4]:
import sqlite3
# this just work by a row number and new trigger price 
# old trigger price for row 34: 166.68
sql  = 'UPDATE option_strategies SET trigger_price=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (66, 71))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)

## Add New column

In [13]:
import sqlite3
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def add_premium_column():
    """Add a 'timestamp_of_order ' column to the option_strategies table."""
    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect('option_strategies.db')
        cursor = conn.cursor()
        
        # Check if the column already exists
        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [col[1] for col in cursor.fetchall()]
        
        if 'timestamp_of_order ' in columns:
            logger.info("Column 'timestamp_of_order  already exists in the table.")
            return
        
        # Add the new column - using REAL type for storing decimal/float values
        cursor.execute("""
            ALTER TABLE option_strategies 
            ADD COLUMN timestamp_of_order DATETIME
        """)
        
        conn.commit()
        logger.info("Successfully added 'timestamp_of_order' column to option_strategies table.")
        
    except sqlite3.Error as e:
        logger.error(f"SQLite error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    add_premium_column()

2025-03-27 10:06:28,587 - INFO - Successfully added 'timestamp_of_order' column to option_strategies table.


## Query a section of the database

In [29]:


start_date = '2025-03-29'
dat = sqlite3.connect('option_strategies.db')
query_text = f"""
    SELECT * FROM option_strategies 
    WHERE date(scrape_date) = '{start_date}'
    AND strategy_status = 'triggered'
    """
query = dat.execute(query_text)
cols = [column[0] for column in query.description]
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
results

,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,...,timestamp_of_price_when_last_checked,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed,premium_when_last_checked,timestamp_of_order
0,72,2025-03-29T06:05:03.195784,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,TSM,170,sell 155.0 - buy 145.0,145.0,155.0,246.0,...,2025-03-29T06:38:14.158376,4,2025-04-17,March,2025-03-29T06:38:14.168108,triggered,168.24,None,None,None
